In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained =  './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/1000shot/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.5, 'drop_connect_rate':0.5})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    if (epoch+1) % 10 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 20/20 | Loss:0.9417 | MainLoss:0.8962 | Alpha:0.0216 | SPLoss:1.4114 | CLSLoss:1.4380 | top1:49.6053 | AUROC:0.0000
Test | 39/20 | Loss:0.8531 | MainLoss:0.6929 | SPLoss:1.4647 | CLSLoss:1.3707 | top1:50.0000 | AUROC:0.5740
Test | 161/20 | Loss:0.8421 | MainLoss:0.6819 | SPLoss:1.4647 | CLSLoss:1.3707 | top1:59.2804 | AUROC:0.9999

Epoch: [2 | 1000] LR: 0.130000
Train | 20/20 | Loss:0.7409 | MainLoss:0.6937 | Alpha:0.0246 | SPLoss:1.3767 | CLSLoss:1.3337 | top1:50.7105 | AUROC:0.0000
Test | 39/20 | Loss:0.8336 | MainLoss:0.6929 | SPLoss:1.2782 | CLSLoss:1.2943 | top1:50.0000 | AUROC:0.5751
Test | 161/20 | Loss:0.8178 | MainLoss:0.6770 | SPLoss:1.2782 | CLSLoss:1.2943 | top1:73.8380 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.160000
Train | 20/20 | Loss:0.7342 | MainLoss:0.6945 | Alpha:0.0229 | SPLoss:1.1910 | CLSLoss:1.2506 | top1:50.0000 | AUROC:0.0000
Test | 39/20 | Loss:0.8146 | MainLoss:0.6931 | SPLoss:1.0953 | CLSLoss:1.2019 | top1:50.0000 | AUR

Train | 20/20 | Loss:3.7409 | MainLoss:0.5088 | Alpha:0.3471 | SPLoss:9.2718 | CLSLoss:0.5813 | top1:76.3947 | AUROC:0.0000
Test | 39/20 | Loss:1.0694 | MainLoss:0.5015 | SPLoss:5.6299 | CLSLoss:0.4957 | top1:77.9487 | AUROC:0.9221
Test | 161/20 | Loss:1.1426 | MainLoss:0.5746 | SPLoss:5.6299 | CLSLoss:0.4957 | top1:72.4019 | AUROC:0.8003

Epoch: [24 | 1000] LR: 0.399858
Train | 20/20 | Loss:2.2691 | MainLoss:0.4983 | Alpha:0.3465 | SPLoss:5.1120 | CLSLoss:0.5678 | top1:76.4211 | AUROC:0.0000
Test | 39/20 | Loss:4.1588 | MainLoss:0.3750 | SPLoss:37.7695 | CLSLoss:0.6854 | top1:83.4103 | AUROC:0.9179
Test | 161/20 | Loss:4.4252 | MainLoss:0.6413 | SPLoss:37.7696 | CLSLoss:0.6854 | top1:68.3489 | AUROC:0.8099

Epoch: [25 | 1000] LR: 0.399833
Train | 20/20 | Loss:21.6828 | MainLoss:0.5665 | Alpha:0.3466 | SPLoss:60.4746 | CLSLoss:0.5442 | top1:71.4211 | AUROC:0.0000
Test | 39/20 | Loss:1.7766 | MainLoss:0.4255 | SPLoss:13.4602 | CLSLoss:0.5131 | top1:83.2308 | AUROC:0.9066
Test | 161/20 |

Train | 20/20 | Loss:0.7242 | MainLoss:0.3407 | Alpha:0.4135 | SPLoss:0.9104 | CLSLoss:0.8963 | top1:85.1053 | AUROC:0.0000
Test | 39/20 | Loss:0.3748 | MainLoss:0.3081 | SPLoss:0.5725 | CLSLoss:0.9447 | top1:87.5256 | AUROC:0.9528
Test | 161/20 | Loss:1.4650 | MainLoss:1.3983 | SPLoss:0.5725 | CLSLoss:0.9447 | top1:53.5109 | AUROC:0.5991

Epoch: [46 | 1000] LR: 0.398860
Train | 20/20 | Loss:0.6214 | MainLoss:0.3713 | Alpha:0.4144 | SPLoss:0.5821 | CLSLoss:0.8712 | top1:83.6053 | AUROC:0.0000
Test | 39/20 | Loss:0.4172 | MainLoss:0.3545 | SPLoss:0.5460 | CLSLoss:0.8101 | top1:84.8846 | AUROC:0.9549
Test | 161/20 | Loss:1.0875 | MainLoss:1.0248 | SPLoss:0.5460 | CLSLoss:0.8101 | top1:50.8505 | AUROC:0.5117

Epoch: [47 | 1000] LR: 0.398792
Train | 20/20 | Loss:0.5533 | MainLoss:0.3532 | Alpha:0.4160 | SPLoss:0.4610 | CLSLoss:0.8273 | top1:84.4211 | AUROC:0.0000
Test | 39/20 | Loss:0.3118 | MainLoss:0.2425 | SPLoss:0.6058 | CLSLoss:0.8723 | top1:90.2949 | AUROC:0.9675
Test | 161/20 | Loss

Train | 20/20 | Loss:0.1919 | MainLoss:0.1483 | Alpha:0.4576 | SPLoss:0.0725 | CLSLoss:1.0436 | top1:94.2368 | AUROC:0.0000
Test | 39/20 | Loss:0.1860 | MainLoss:0.1683 | SPLoss:0.0720 | CLSLoss:1.0469 | top1:93.4487 | AUROC:0.9839
Test | 161/20 | Loss:1.4311 | MainLoss:1.4135 | SPLoss:0.0720 | CLSLoss:1.0469 | top1:54.9190 | AUROC:0.6270

Epoch: [68 | 1000] LR: 0.039691
Train | 20/20 | Loss:0.1859 | MainLoss:0.1420 | Alpha:0.4596 | SPLoss:0.0727 | CLSLoss:1.0500 | top1:94.6842 | AUROC:0.0000
Test | 39/20 | Loss:0.1989 | MainLoss:0.1810 | SPLoss:0.0744 | CLSLoss:1.0528 | top1:92.9615 | AUROC:0.9836
Test | 161/20 | Loss:1.4170 | MainLoss:1.3990 | SPLoss:0.0744 | CLSLoss:1.0528 | top1:55.5296 | AUROC:0.6247

Epoch: [69 | 1000] LR: 0.039680
Train | 20/20 | Loss:0.1897 | MainLoss:0.1449 | Alpha:0.4596 | SPLoss:0.0745 | CLSLoss:1.0539 | top1:94.8421 | AUROC:0.0000
Test | 39/20 | Loss:0.1871 | MainLoss:0.1690 | SPLoss:0.0759 | CLSLoss:1.0553 | top1:93.4615 | AUROC:0.9838
Test | 161/20 | Loss

Train | 20/20 | Loss:0.1325 | MainLoss:0.0982 | Alpha:0.4756 | SPLoss:0.0502 | CLSLoss:1.0445 | top1:96.6579 | AUROC:0.0000
Test | 39/20 | Loss:0.1616 | MainLoss:0.1460 | SPLoss:0.0522 | CLSLoss:1.0428 | top1:94.5641 | AUROC:0.9887
Test | 161/20 | Loss:1.7129 | MainLoss:1.6972 | SPLoss:0.0522 | CLSLoss:1.0428 | top1:54.8100 | AUROC:0.6372

Epoch: [90 | 1000] LR: 0.039403
Train | 20/20 | Loss:0.1369 | MainLoss:0.1001 | Alpha:0.4764 | SPLoss:0.0554 | CLSLoss:1.0438 | top1:96.2105 | AUROC:0.0000
Test | 39/20 | Loss:0.1584 | MainLoss:0.1423 | SPLoss:0.0569 | CLSLoss:1.0386 | top1:94.6923 | AUROC:0.9882
Test | 161/20 | Loss:1.7360 | MainLoss:1.7199 | SPLoss:0.0569 | CLSLoss:1.0386 | top1:54.2835 | AUROC:0.6278

Epoch: [91 | 1000] LR: 0.039387
Train | 20/20 | Loss:0.1060 | MainLoss:0.0897 | Alpha:0.4814 | SPLoss:0.0124 | CLSLoss:1.0410 | top1:97.0000 | AUROC:0.0000
Test | 39/20 | Loss:0.1524 | MainLoss:0.1395 | SPLoss:0.0238 | CLSLoss:1.0447 | top1:94.9231 | AUROC:0.9896
Test | 161/20 | Loss

Train | 20/20 | Loss:0.0884 | MainLoss:0.0728 | Alpha:0.4865 | SPLoss:0.0119 | CLSLoss:0.9863 | top1:97.4737 | AUROC:0.0000
Test | 39/20 | Loss:0.1446 | MainLoss:0.1320 | SPLoss:0.0275 | CLSLoss:0.9844 | top1:95.4231 | AUROC:0.9911
Test | 161/20 | Loss:1.9612 | MainLoss:1.9486 | SPLoss:0.0275 | CLSLoss:0.9844 | top1:54.5919 | AUROC:0.6520

Epoch: [112 | 1000] LR: 0.039021
Train | 20/20 | Loss:0.0961 | MainLoss:0.0709 | Alpha:0.4858 | SPLoss:0.0316 | CLSLoss:0.9802 | top1:97.5789 | AUROC:0.0000
Test | 39/20 | Loss:0.1609 | MainLoss:0.1479 | SPLoss:0.0315 | CLSLoss:0.9807 | top1:95.1026 | AUROC:0.9914
Test | 161/20 | Loss:1.8993 | MainLoss:1.8863 | SPLoss:0.0315 | CLSLoss:0.9807 | top1:55.3115 | AUROC:0.6535

Epoch: [113 | 1000] LR: 0.039002
Train | 20/20 | Loss:0.0957 | MainLoss:0.0687 | Alpha:0.4862 | SPLoss:0.0354 | CLSLoss:0.9790 | top1:97.4737 | AUROC:0.0000
Test | 39/20 | Loss:0.1425 | MainLoss:0.1286 | SPLoss:0.0413 | CLSLoss:0.9775 | top1:95.5769 | AUROC:0.9913
Test | 161/20 | Lo

Train | 20/20 | Loss:0.0771 | MainLoss:0.0522 | Alpha:0.4897 | SPLoss:0.0324 | CLSLoss:0.8983 | top1:98.1579 | AUROC:0.0000
Test | 39/20 | Loss:0.1433 | MainLoss:0.1314 | SPLoss:0.0297 | CLSLoss:0.9002 | top1:95.6026 | AUROC:0.9921
Test | 161/20 | Loss:2.1361 | MainLoss:2.1241 | SPLoss:0.0297 | CLSLoss:0.9002 | top1:54.1028 | AUROC:0.6441

Epoch: [134 | 1000] LR: 0.038549
Train | 20/20 | Loss:0.0690 | MainLoss:0.0428 | Alpha:0.4921 | SPLoss:0.0350 | CLSLoss:0.9011 | top1:98.6316 | AUROC:0.0000
Test | 39/20 | Loss:0.1543 | MainLoss:0.1414 | SPLoss:0.0381 | CLSLoss:0.9055 | top1:95.3718 | AUROC:0.9923
Test | 161/20 | Loss:2.0211 | MainLoss:2.0082 | SPLoss:0.0381 | CLSLoss:0.9055 | top1:55.5826 | AUROC:0.6575

Epoch: [135 | 1000] LR: 0.038525
Train | 20/20 | Loss:0.0739 | MainLoss:0.0475 | Alpha:0.4906 | SPLoss:0.0355 | CLSLoss:0.9050 | top1:98.5263 | AUROC:0.0000
Test | 39/20 | Loss:0.1593 | MainLoss:0.1468 | SPLoss:0.0344 | CLSLoss:0.9045 | top1:95.2821 | AUROC:0.9924
Test | 161/20 | Lo

Train | 20/20 | Loss:0.0657 | MainLoss:0.0402 | Alpha:0.4913 | SPLoss:0.0351 | CLSLoss:0.8267 | top1:98.6316 | AUROC:0.0000
Test | 39/20 | Loss:0.1413 | MainLoss:0.1294 | SPLoss:0.0358 | CLSLoss:0.8268 | top1:95.8205 | AUROC:0.9934
Test | 161/20 | Loss:2.1678 | MainLoss:2.1560 | SPLoss:0.0358 | CLSLoss:0.8268 | top1:53.8193 | AUROC:0.6293

Epoch: [156 | 1000] LR: 0.037988
Train | 20/20 | Loss:0.0695 | MainLoss:0.0448 | Alpha:0.4910 | SPLoss:0.0337 | CLSLoss:0.8258 | top1:98.3684 | AUROC:0.0000
Test | 39/20 | Loss:0.1345 | MainLoss:0.1224 | SPLoss:0.0380 | CLSLoss:0.8230 | top1:95.9359 | AUROC:0.9939
Test | 161/20 | Loss:2.1789 | MainLoss:2.1669 | SPLoss:0.0380 | CLSLoss:0.8230 | top1:53.5296 | AUROC:0.6331

Epoch: [157 | 1000] LR: 0.037961
Train | 20/20 | Loss:0.0696 | MainLoss:0.0426 | Alpha:0.4907 | SPLoss:0.0384 | CLSLoss:0.8243 | top1:98.5789 | AUROC:0.0000
Test | 39/20 | Loss:0.1300 | MainLoss:0.1181 | SPLoss:0.0371 | CLSLoss:0.8245 | top1:96.0769 | AUROC:0.9933
Test | 161/20 | Lo

Train | 20/20 | Loss:0.0532 | MainLoss:0.0333 | Alpha:0.4927 | SPLoss:0.0247 | CLSLoss:0.7703 | top1:99.0263 | AUROC:0.0000
Test | 39/20 | Loss:0.1366 | MainLoss:0.1264 | SPLoss:0.0240 | CLSLoss:0.7709 | top1:95.8846 | AUROC:0.9937
Test | 161/20 | Loss:2.2754 | MainLoss:2.2653 | SPLoss:0.0240 | CLSLoss:0.7709 | top1:53.6012 | AUROC:0.6368

Epoch: [178 | 1000] LR: 0.037341
Train | 20/20 | Loss:0.0714 | MainLoss:0.0498 | Alpha:0.4907 | SPLoss:0.0284 | CLSLoss:0.7680 | top1:98.0000 | AUROC:0.0000
Test | 39/20 | Loss:0.1209 | MainLoss:0.1096 | SPLoss:0.0369 | CLSLoss:0.7628 | top1:96.1923 | AUROC:0.9939
Test | 161/20 | Loss:2.3151 | MainLoss:2.3038 | SPLoss:0.0369 | CLSLoss:0.7628 | top1:52.9470 | AUROC:0.6381

Epoch: [179 | 1000] LR: 0.037310
Train | 20/20 | Loss:0.0692 | MainLoss:0.0456 | Alpha:0.4924 | SPLoss:0.0323 | CLSLoss:0.7646 | top1:98.4211 | AUROC:0.0000
Test | 39/20 | Loss:0.1255 | MainLoss:0.1142 | SPLoss:0.0374 | CLSLoss:0.7586 | top1:96.2051 | AUROC:0.9934
Test | 161/20 | Lo

KeyboardInterrupt: 